# Training the reader on SQuAD FR dataset

This notebook shows how to fine-tune a pre-trained BERT model on the SQuAD.

***Original CDQA Note:*** *To run this notebook you will need to have access to GPU. The fine-tuning of the Reader was done with an AWS EC2 p3.2xlarge machine (GPU Tesla V100 16GB). It took about 2 hours to complete (2 epochs on SQuAD 1.1 train was enough to achieve SOTA results on SQuAD 1.1 dev).*

In [1]:
import torch
import joblib
import json
import subprocess
import pandas as pd
from bertqa_sklearn_fr import BertProcessor, BertQA
import re, os

D:\Etalab\demo-piaf\bertqa_sklearn_fr.py:35: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Check SQuAD FR dataset

In [2]:
input_file = './data/SQuAD_FR/annotations-18112019.json'

In [3]:
with open(input_file) as json_file:
    d = json.load(json_file)

In [1]:
# d[0]['paragraphs'][0]['questions']

In [2]:
print(len(d))

NameError: name 'd' is not defined

### Preprocess SQuAD examples

In [5]:
train_processor = BertProcessor(bert_model='bert-base-multilingual-uncased', do_lower_case=True, is_training=True)
train_examples, train_features = train_processor.fit_transform(X=input_file)

### Train the model

In [7]:
reader = BertQA(train_batch_size=12,
                learning_rate=3e-5,
                num_train_epochs=2,
                do_lower_case=True,
                output_dir='models')

In [8]:
# My GPU doesn't have engough memory (total 2GB), but comment this to use GPU instead of CPU
reader.model.to('cpu')
reader.device = torch.device('cpu')

In [9]:
reader.fit(X=(train_examples, train_features))

BertQA(bert_model='bert-base-uncased', do_lower_case=True, fp16=False,
       gradient_accumulation_steps=1, learning_rate=3e-05, local_rank=-1,
       loss_scale=0, max_answer_length=30, n_best_size=20, no_cuda=False,
       null_score_diff_threshold=0.0, num_train_epochs=2, output_dir='models',
       predict_batch_size=8, seed=42, server_ip='', server_port='',
       train_batch_size=12, verbose_logging=False,
       version_2_with_negative=False, warmup_proportion=0.1)

### Save model locally

In [10]:
joblib.dump(reader, os.path.join(reader.output_dir, 'bert_qa_fr.joblib'))

['models\\bert_qa_fr.joblib']